In [1]:
import sys

sys.path.insert(0, '..')

import os
import requests
import json
from tempfile import mkdtemp
import libsbml
import amici
import os
from biosimulator_processes.utils import prepare_single_ode_process_document
from biosimulator_processes.data_model.compare_data_model import ODEComparisonDocument, DocumentFactory
from biosimulator_processes import CORE
from process_bigraph import Composite, pp

In [3]:
#### Step 1: Define the document to be read by the Composite, which is implemented by the Process.
biomodel_id = 'BIOMD0000000630'
model_fp = f'../biosimulator_processes/model_files/sbml/{biomodel_id}_url.xml'
species_context = 'concentrations'
species_port_name = f'floating_species_{species_context}'
species_store = [f'floating_species_{species_context}_store']
duration = 30

comparison_document = ODEComparisonDocument(
    simulators=['copasi', 'amici'],
    duration=20,
    num_steps=32,
    model_filepath=model_fp)

comparison_document.composite

In [3]:
# Amici Model from compiled module
sbml_importer = amici.SbmlImporter(model_fp)
model_name = biomodel_id
model_dir = "./model_dir"
if not os.path.exists(model_dir):
    os.mkdir(model_dir)

sbml_importer.sbml2amici(model_name, model_dir)
# load the model module
model_module = amici.import_model_module(model_name, model_dir)
# instantiate model
compiled_model = model_module.getModel()
# instantiate solver
method = compiled_model.getSolver()

In [4]:
# From load sbml model module 
sbml_reader = libsbml.SBMLReader()
sbml_doc = sbml_reader.readSBML(model_fp)
sbml_model = sbml_doc.getModel()
species_objects_list = sbml_model.getListOfSpecies()
floating_species_list = [s.getId() for s in species_objects_list]
floating_species_initial = dict(zip(floating_species_list, [s.getInitialConcentration() for s in species_objects_list]))

In [24]:
for s in species_objects_list:
    print(s.getInitialAmount())

In [19]:
[s.getId() for s in species_objects_list]

In [18]:
floating_species_initial

In [12]:
dict(zip(floating_species_list, [s.getInitialConcentration() for s in species_objects_list]))

In [4]:
floating_species_initial

In [18]:
species_objects_list = sbml_model.getListOfSpecies()

In [23]:
for i, s in enumerate(species_objects_list):
    print(s.getId())
    print(i, s.getInitialConcentration())

In [25]:
dir(model)

In [26]:
model.getParameters()

In [27]:
model.getParameterList()

In [28]:
model.getParameterIds()

In [ ]:
model_parameters_list = model.getP

In [29]:
dir(sbml_model)

In [36]:
model_parameter_objects = sbml_model.getListOfParameters()

In [37]:
model_parameter_objects

In [38]:
for m in model_parameter_objects:
    print(dir(m))
    print(m.getValue())
    print(m.getName())

In [ ]:
model_parameter_objects = sbml_model.getListOfParameters()
model_parameters_list = [param.getName() for param in model_parameter_objects]
model_parameters_values = [param.getValue() for param in model_parameter_objects]

In [39]:
dir(sbml_model)

In [50]:
reaction_objects = sbml_model.reactions
reaction_list = [reaction.getName() for reaction in reaction_objects]

reaction_list

In [45]:
dir(reaction_objects[0])

In [49]:
reaction_objects[0].getProduct()

In [8]:
dir(sbml_model)

In [4]:
from biosimulator_processes.processes.amici_process import AmiciProcess

simulator_to_use = 'amici'
process_id = 'simple_' + simulator_to_use
amici_process_document = prepare_single_ode_process_document(
    process_id=process_id,
    simulator_name=simulator_to_use,
    sbml_model_fp=model_fp)

amici_process_document

In [5]:
amici_process_config = amici_process_document[process_id]['config']

In [6]:
amici_process_config

In [7]:
amici_workflow = Composite(
    config={'state': amici_process_document},
    core=CORE)

In [13]:
amici_process: AmiciProcess = amici_workflow.state[process_id]['instance']

In [14]:
amici_process.initial_state()

In [15]:
amici_process.reaction_list

In [26]:
amici_process.initial_state()

In [23]:
# Let's compare that against copasi process
from biosimulator_processes.processes.copasi_process import CopasiProcess


copasi_process: CopasiProcess = CopasiProcess(
    config=comparison_document.composite['copasi_0']['config'])

In [32]:
copasi_process.outputs() == amici_process.outputs()

In [33]:
amici_process.initial_state() == copasi_process.initial_state()

In [34]:
copasi_process.reaction_list == amici_process.reaction_list